In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle

In [3]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.33
set_session(tf.Session(config=config))

/home/david/anaconda3/envs/CN_AQ/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
Urban_station_lonlats = np.array([
['dongsi_aq',116.417,39.929],
['tiantan_aq',116.407,39.886],
['guanyuan_aq',116.339,39.929],
['wanshouxigong_aq',116.352,39.878],
['aotizhongxin_aq',116.397,39.982],
['nongzhanguan_aq',116.461,39.937],
['wanliu_aq',116.287,39.987],
['beibuxinqu_aq',116.174,40.09],
['zhiwuyuan_aq',116.207,40.002],
['fengtaihuayuan_aq',116.279,39.863],
['yungang_aq',116.146,39.824],
['gucheng_aq',116.184,39.914]
])
lons = np.array(Urban_station_lonlats[:,1],dtype = np.float32)
lats = np.array(Urban_station_lonlats[:,2],dtype = np.float32)
print(lons.max(),lons.min(),lons.mean())
print(lats.max(),lats.min(),lats.mean())
X_length = 93.6
Y_length = 41.2
#Change X Y to degree
Y_degree = Y_length / 111
#Consider the length of the latitude line
X_degree = X_length / (40000 * np.cos(lats.mean() / 180 * 3.1415926) / 360)
#X_degree = X_length / 111

116.461 116.146 116.30417
40.09 39.824 39.935085


In [5]:
lat_step = np.arange(lats.mean() - Y_degree,lats.mean() + Y_degree,.1)
lon_step = np.arange(lons.mean() - X_degree,lons.mean() + X_degree,.1)
#Build the step map
for index in range(len(lat_step)):
    lat_step[index] = round(lat_step[index],1)
for index in range(len(lon_step)):
    lon_step[index] = round(lon_step[index],1)
print(lat_step)
print(lon_step)
#Get the size of point waiting to be analyzed
point_size = len(lat_step) * len(lon_step)
with open("lat_lon_range.pkl","wb") as file:
    pickle.dump((lat_step,lon_step),file)
print("ok")

[39.6 39.7 39.8 39.9 40.  40.1 40.2 40.3]
[115.2 115.3 115.4 115.5 115.6 115.7 115.8 115.9 116.  116.1 116.2 116.3
 116.4 116.5 116.6 116.7 116.8 116.9 117.  117.1 117.2 117.3]
ok


In [6]:
#Use panda to read csv
beijing_weather_data = pd.read_csv("Beijing_historical_meo_grid.csv")
print(beijing_weather_data.head())
#Get the value of beijing data
beijing_weather_data_value = beijing_weather_data.values

        stationName  longitude  latitude             utc_time  temperature  \
0  beijing_grid_000      115.0      39.0  2017-01-01 00:00:00        -5.47   
1  beijing_grid_001      115.0      39.1  2017-01-01 00:00:00        -5.53   
2  beijing_grid_002      115.0      39.2  2017-01-01 00:00:00        -5.70   
3  beijing_grid_003      115.0      39.3  2017-01-01 00:00:00        -5.88   
4  beijing_grid_004      115.0      39.4  2017-01-01 00:00:00        -5.34   

   pressure  humidity  wind_direction  wind_speed/kph  
0    984.73     76.60           53.71            3.53  
1    979.33     75.40           43.59            3.11  
2    963.14     71.80            0.97            2.75  
3    946.94     68.20          327.65            3.84  
4    928.80     58.81          317.85            6.14  


In [ ]:
from datetime import date
from datetime import datetime
#Build the datetime object
for index in tqdm(range(len(beijing_weather_data_value))):
    beijing_weather_data_value[index][3] = datetime.strptime(
        beijing_weather_data_value[index][3],'%Y-%m-%d %H:%M:%S')

In [ ]:
#Caculate the days between June 31 to March 1
start = date(2017,3,1)
end = date(2017,7,1)
#Set the correct array size
collection_size = point_size * (end - start).days * 24
collection = [None] * collection_size
#Select the data from March to June
def get_Data(beijing_weather_data_value,collection,collection_size):
    print(collection_size)
    cursor = 0
    for index in tqdm(range(beijing_weather_data_value.shape[0])):
        for lat in lat_step:
            for lon in lon_step:
                if (beijing_weather_data_value[index][1] == lon and 
                    beijing_weather_data_value[index][2] == lat and
                    beijing_weather_data_value[index][3].month >= 3 and
                    beijing_weather_data_value[index][3].month <= 6):
                    collection[cursor] = beijing_weather_data_value[index]
                    cursor = cursor + 1
                    if cursor == collection_size:
                        print(cursor)
                        return collection;
#Extract all the data from CSV
collection = get_Data(beijing_weather_data_value,collection,collection_size)
with open("1hour.pkl","wb") as file:
    pickle.dump(collection, file, protocol=pickle.HIGHEST_PROTOCOL)
print('ok!')

In [8]:
#Read the data
import pickle
collection = list()
with open("1hour.pkl","rb") as file:
    collection = pickle.load(file)
print(collection[0])
#Read height data
with open("All_height_data_1hr.pk","rb") as file:
    height_data = pickle.load(file)
#print(round(height_data[-1]['latitude'],1))
print(height_data[-1])

['beijing_grid_048' 115.2 39.6 datetime.datetime(2017, 3, 1, 0, 0) -4.4
 916.13 35.47 333.37 22.56]
{'_id': ObjectId('5ae019a950fc202dcd53e02f'), 'longitude': 115.29411764705883, 'latitude': 39.986486486486484, 'elevation': 1600.509399414062}


In [9]:
lat_step_dict = {lat_step[i]:i for i in range(len(lat_step))}
print(len(lat_step_dict))
lon_step_dict = {lon_step[i]:i for i in range(len(lon_step))}
print(len(lon_step_dict))
print(lat_step_dict)
Height_map = np.zeros(shape = (len(lon_step),len(lat_step)))
for data in height_data:
    lat_index = lat_step_dict[round(data['latitude'],1)]
    lon_index = lon_step_dict[round(data['longitude'],1)]
    if not Height_map[lon_index][lat_index] == 0:
        Height_map[lon_index][lat_index] = (
            Height_map[lon_index][lat_index] + data['elevation'])
    else:
        Height_map[lon_index][lat_index] = data['elevation']
#print(Height_map)

8
22
{39.6: 0, 39.7: 1, 39.8: 2, 39.9: 3, 40.0: 4, 40.1: 5, 40.2: 6, 40.3: 7}


In [17]:
#Prepare datas for SVR
import pickle
from datetime import datetime
print(collection[0][3])
#collection[:,3] = datetime.strptime('%y-%m-%d %H:%M:%S',collection[:,3])
lon = np.zeros(shape = (len(collection)))
lat = np.zeros(shape = (len(collection)))
month = np.zeros(shape = (len(collection)))
date = np.zeros(shape = (len(collection)))
hour = np.zeros(shape = (len(collection)))
temp = np.zeros(shape = (len(collection)))
humid = np.zeros(shape = (len(collection)))
press = np.zeros(shape = (len(collection)))
direct = np.zeros(shape = (len(collection)))
speed = np.zeros(shape = (len(collection)))
for index in tqdm(range(len(collection))):
    lon[index] = collection[index][1]
    lat[index] = collection[index][2]
    month[index] = collection[index][3].month
    date[index] = collection[index][3].day
    hour[index] = collection[index][3].hour
    temp[index] = collection[index][4]
    humid[index] = collection[index][6]
    press[index] = collection[index][5]
    direct[index] = collection[index][7]
    speed[index] = collection[index][8]
lon = np.expand_dims(lon,axis = 1)
lat = np.expand_dims(lat,axis = 1)
month = np.expand_dims(month,axis = 1)
date = np.expand_dims(date,axis = 1)
hour = np.expand_dims(hour,axis = 1)
temp = np.expand_dims(temp,axis = 1)
humid = np.expand_dims(humid,axis = 1)
press = np.expand_dims(press,axis = 1)
direct = np.expand_dims(direct,axis = 1)
speed = np.expand_dims(speed,axis = 1)
X = np.concatenate((lon,lat,month,date,hour,temp,humid,press,direct,speed),axis = 1)
with open("1hour_prepared.pkl","wb") as file:
    pickle.dump(X, file, protocol=pickle.HIGHEST_PROTOCOL)

  5%|▌         | 26849/515328 [00:00<00:01, 268378.92it/s]

2017-03-01 00:00:00


100%|██████████| 515328/515328 [00:01<00:00, 315564.63it/s]


(515328, 10)


In [6]:
#Deserialize all the raw grid data
import pickle
with open("1hour_prepared.pkl","rb") as file:
    X = pickle.load(file)
print(X[:10])
print(X.shape)
'''0.經度(longt)
1.緯度(lat)
2.月份(month)
3.小時(hour)
4.溫度(temp)
5.濕度(humidity)
6.氣壓(pressure)
7.風向(wind_direction)
8.風速(wind_speed)'''

[[115.2   39.6    3.     1.     0.    -4.4   35.47 916.13 333.37  22.56]
 [115.2   39.7    3.     1.     0.    -5.79  39.06 905.54 332.07  25.16]
 [115.2   39.8    3.     1.     0.    -7.14  42.96 892.6  330.67  26.78]
 [115.2   39.9    3.     1.     0.    -8.44  47.44 874.98 328.6   26.46]
 [115.2   40.     3.     1.     0.    -9.73  51.93 857.35 326.49  26.17]
 [115.2   40.1    3.     1.     0.    -8.61  48.58 880.25 324.21  27.03]
 [115.2   40.2    3.     1.     0.    -7.48  45.24 903.14 322.07  27.92]
 [115.2   40.3    3.     1.     0.    -6.74  42.42 917.75 322.45  28.27]
 [115.3   39.6    3.     1.     0.    -2.54  32.41 939.33 337.13  17.22]
 [115.3   39.7    3.     1.     0.    -3.58  33.74 938.19 334.49  21.71]]
(515328, 10)


'0.經度(longt)\n1.緯度(lat)\n2.月份(month)\n3.小時(hour)\n4.溫度(temp)\n5.濕度(humidity)\n6.氣壓(pressure)\n7.風向(wind_direction)\n8.風速(wind_speed)'

# SVR analysis

In [ ]:
#SVR analysis
'''X => SVR輸入向量的numpy array，資料必須照著下面的順序:
0.經度(longt)
1.緯度(lat)
2.月份(month)
3.天數(day)
4.小時(hour)
5.溫度(temp)
6.濕度(humidity)
7.氣壓(pressure)
8.風向(wind_direction)
9.風速(wind_speed)
Y => PM2.5的預測值'''
import pickle
from sklearn.svm import SVR
from sklearn.externals import joblib
import numpy as np

svr = joblib.load('SVR_beijing.pkl')
X = np.array(X, dtype=np.float64)
X_mu = np.array([116.40616279,39.98908696,4.45972151,14.88423765,11.40249715,17.0021192,33.65642229,996.1066705,208.34196307,10.85087688])
X_sigma = np.array([0.28161015,0.22815651,1.14854307,8.88446275,6.90495716,8.44671392,19.41044621,19.83802731,96.65489494,6.85870324])
Y_mu = 59.541867360598516
Y_sigma = 61.21714698899477

Xs = (X - X_mu) / X_sigma #對要輸入的X向量標準化
Y_pdt = svr.predict(Xs) #將標準化後的X向量放入SVR預測
Y = Y_pdt * Y_sigma + Y_mu #對SVR得到的預測值反標準化
print(Y[:10])

In [ ]:
import pickle as pk
with open("1_hr_pm25.pkl","wb") as file:
    pk.dump(Y, file, protocol=pickle.HIGHEST_PROTOCOL)

# Preprocessing

In [10]:
#Normalize input data
def normalize(x):
    norm = (x - x.min())/(x.max() - x.min())
    return norm

In [11]:
import pickle as pk
import numpy as np
from datetime import date
#load all the prepared data for SVR(to add feature to )
with open("1hour_prepared.pkl","rb") as file:
    X = pk.load(file)
#load all the SVR data
with open("1_hr_pm25.pkl","rb") as file:
    Y = pk.load(file)
#load the ercise point of latitude and longitude
with open("lat_lon_range.pkl","rb") as file:
    lat_step, lon_step = pk.load(file)
#Y transform to [date][lon][lat]
date_count = (date(2017,7,1) - date(2017,3,1)).days
Y = np.reshape(Y, (24 * date_count,len(lon_step),len(lat_step),1))
Y[:] = normalize(Y[:])
Height_map = np.expand_dims(Height_map,axis = 0)
Height_map = np.repeat(Height_map,24*date_count,axis = 0)
Height_map = np.expand_dims(Height_map,axis = 3)
print(Height_map.shape)
#Get all the other feature
temp = np.reshape(X[:,5],Y.shape)
print(temp.shape)
humidity = np.reshape(X[:,6],Y.shape)
pressure = np.reshape(X[:,7],Y.shape)
wind_speed = np.reshape(X[:,9],Y.shape)
temp[:] = normalize(temp[:])
humidity[:] = normalize(humidity[:])
pressure[:] = normalize(pressure[:])
wind_speed[:] = normalize(wind_speed[:])
Height_map[:] = normalize(Height_map[:])
Y = np.concatenate((Y,temp,humidity,pressure,wind_speed,Height_map),axis = 3)
print(Y.shape)
#Get the longitude of the target site
target_lon = np.argwhere(lon_step == 116.3)
#Get the latitude of the target site
target_lat = np.argwhere(lat_step == 39.9)
#There  are 256 objects in pickle file and there are 10 pickle files
validation_size = len(Y) - 2560 
#Dump validation data
with open("1hour_pm25_validation_array.pkl","wb") as file:
    pk.dump((Y[-validation_size - 1:-1],Y[-validation_size:,target_lon,target_lat,0]),file)
#Dump testing data
#Assume that batch size is 256
for index in range(1, int(len(Y) / 256)):
    with open("batch_number_" + str(index) + ".pkl","wb") as file:
        if 256 * index >= len(Y):
            end_batch = len(Y) - 1
        else:
            end_batch = 256 * index
        pk.dump((Y[256 * (index - 1):end_batch],#Testing batch
                 Y[256 * (index - 1) + 1:end_batch + 1 ,target_lon,target_lat,0]),file)#Testing label
print("ok!")

(2928, 22, 8, 1)
(2928, 22, 8, 1)
(2928, 22, 8, 6)
ok!


# Start building network

In [27]:
import tensorflow as tf
import numpy as np
import pickle as pk
import os

In [28]:
#print(validation_size)
batch = list()
batch_label = list()
#Serialize all the batch
for index in range(1,11):
    with open("batch_number_" + str(index) + ".pkl","rb") as file:
        batch_read, batch_label_read = pk.load(file)
        batch.append(batch_read)
        batch_label.append(batch_label_read)
#Serialize the area 
with open("lat_lon_range.pkl","rb") as file:
    lat_step, lon_step = pk.load(file)

with open("1hour_pm25_validation_array.pkl","rb") as file:
    X_validation, Y_validation = pk.load(file)
print("================")
print("Validation Shape")
print(X_validation.shape)
print(Y_validation.shape)
print("================")
#print(X_validation[-1])
#print(Y_validation[-2])
batch = np.array(batch)
batch_label = np.array(batch_label)
print("Testing Shape")
print(batch.shape)
print(batch_label.shape)
print("================")

Validation Shape
(368, 22, 8, 6)
(368, 1, 1)
Testing Shape
(10, 256, 22, 8, 6)
(10, 256, 1, 1)


In [29]:
def CNN_layer(x_tensor, conv_num_outputs, conv_ksize, conv_strides):
    x_shape = x_tensor.get_shape().as_list()
    weights = tf.Variable(tf.truncated_normal(
        [*conv_ksize[:],x_shape[-1],conv_num_outputs],
        mean = 0, stddev = 0.1))
    strides = [1,*conv_strides[:],1]
    padding = "SAME"
    convolution = tf.nn.conv2d(x_tensor,weights,strides,padding)
    #No batch norm
    #convolution = tf.layers.batch_normalization(convolution, training=training)
    convolution = tf.nn.relu(convolution)
    print("--------------")
    print("conv_2d")
    print(convolution.get_shape().as_list())
    tf.summary.histogram("weights",weights)
    return convolution

In [30]:
def Build_CNN(x):
    with tf.name_scope("CNN_1"):
        CNN = tf.layers.conv2d(inputs = x,filters = 64,kernel_size = 3,padding = 'SAME',activation = tf.nn.relu)
        CNN = tf.nn.relu(CNN)
        #CNN = CNN_layer(x,64,(3,3),(1,1))
    with tf.name_scope("CNN_2"):
        CNN = tf.layers.conv2d(inputs = x,filters = 64,kernel_size = 3,padding = 'SAME',activation = tf.nn.relu)
        CNN = tf.nn.relu(CNN)
        #CNN = CNN_layer(x,64,(3,3),(1,1))
    with tf.name_scope("flatten"):
        CNN = tf.contrib.layers.flatten(CNN)
    with tf.name_scope("FC1"):
        CNN = tf.contrib.layers.fully_connected(CNN,128)
        CNN = tf.nn.relu(CNN)
    with tf.name_scope("FC2"):
        CNN = tf.contrib.layers.fully_connected(CNN,128)
        CNN = tf.nn.relu(CNN)
    with tf.name_scope("output"):
        CNN = tf.contrib.layers.fully_connected(CNN,1)
    return CNN

In [31]:
def print_loss(session,validation_batch,validation_label,loss):
    loss = session.run(loss,feed_dict = {x:validation_batch,y:validation_label})
    print("|loss:{:.7f}".format(loss),"|")
    pass

In [32]:
#Reset the graph
tf.reset_default_graph()
#Input of the network
x = tf.placeholder(tf.float32,shape = (None,len(lon_step),len(lat_step),6), name = 'x')
y = tf.placeholder(tf.float32,shape = (None),name = 'y')
#Build the network
logits = Build_CNN(x)
# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

#Define Loss and optimizer
with tf.name_scope("loss"):
    a = tf.abs(tf.subtract(y,logits))
    b = tf.add(y,logits)
    loss = tf.scalar_mul(2, tf.reduce_mean(tf.divide(a, b)))
    #loss = tf.losses.mean_squared_error(y,logits)
tf.summary.scalar("loss",loss)
#MSE version 
#loss = tf.sqrt(tf.reduce_mean(tf.losses.mean_squared_error(labels = y,predictions = logits)))
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer().minimize(loss)
merged = tf.summary.merge_all()


In [33]:
epoch = 10000
with tf.Session() as sess:
    writer = tf.summary.FileWriter("log/pm25+temp", graph = sess.graph)
    sess.run(tf.global_variables_initializer())
    for e in range(epoch):
        for batch_num in range(0,10):
            sess.run(optimizer,feed_dict = {x:batch[batch_num],y:batch_label[batch_num]})
            #print("logis:")
            #print(logits.eval({x:X_validation}))
            #print("Y:")
            #print(Y_validation)
            loss_value = sess.run(loss,feed_dict = {x:X_validation,y:Y_validation})
        if e % 30 == 0:
            result = sess.run(merged,feed_dict = {x:X_validation,y:Y_validation})
            writer.add_summary(result,e)
            print("|epoch:",e,"|loss:",loss_value,"|")

|epoch: 0 |loss: 0.54659116 |
|epoch: 30 |loss: 0.19722651 |
|epoch: 60 |loss: 0.18983807 |
|epoch: 90 |loss: 0.1880096 |
|epoch: 120 |loss: 0.19704884 |
|epoch: 150 |loss: 0.19665824 |
|epoch: 180 |loss: 0.19590442 |
|epoch: 210 |loss: 0.19971833 |
|epoch: 240 |loss: 0.18798965 |
|epoch: 270 |loss: 0.19469532 |
|epoch: 300 |loss: 0.18653558 |
|epoch: 330 |loss: 0.18934838 |
|epoch: 360 |loss: 0.1934515 |
|epoch: 390 |loss: 0.18589643 |
|epoch: 420 |loss: 0.1903142 |
|epoch: 450 |loss: 0.19042705 |
|epoch: 480 |loss: 0.18864582 |
|epoch: 510 |loss: 0.18995896 |
|epoch: 540 |loss: 0.18781576 |
|epoch: 570 |loss: 0.18939564 |
|epoch: 600 |loss: 0.19430941 |
|epoch: 630 |loss: 0.19370644 |
|epoch: 660 |loss: 0.19492406 |
|epoch: 690 |loss: 0.19028418 |
|epoch: 720 |loss: 0.18900067 |
|epoch: 750 |loss: 0.19516858 |
|epoch: 780 |loss: 0.19192082 |
|epoch: 810 |loss: 0.19551618 |
|epoch: 840 |loss: 0.19677356 |
|epoch: 870 |loss: 0.20082249 |
|epoch: 900 |loss: 0.19098744 |
|epoch: 930 |los

|epoch: 7530 |loss: 0.18973465 |
|epoch: 7560 |loss: 0.18899208 |
|epoch: 7590 |loss: 0.19047569 |
|epoch: 7620 |loss: 0.19021532 |
|epoch: 7650 |loss: 0.19082408 |
|epoch: 7680 |loss: 0.19000405 |
|epoch: 7710 |loss: 0.19006136 |
|epoch: 7740 |loss: 0.19004521 |
|epoch: 7770 |loss: 0.19006303 |
|epoch: 7800 |loss: 0.19050159 |
|epoch: 7830 |loss: 0.19066323 |
|epoch: 7860 |loss: 0.19073258 |
|epoch: 7890 |loss: 0.190693 |
|epoch: 7920 |loss: 0.18979037 |
|epoch: 7950 |loss: 0.18893199 |
|epoch: 7980 |loss: 0.19075042 |
|epoch: 8010 |loss: 0.19106726 |
|epoch: 8040 |loss: 0.19066676 |
|epoch: 8070 |loss: 0.19112007 |
|epoch: 8100 |loss: 0.19105421 |
|epoch: 8130 |loss: 0.19093134 |
|epoch: 8160 |loss: 0.1909946 |
|epoch: 8190 |loss: 0.19084622 |
|epoch: 8220 |loss: 0.19098693 |
|epoch: 8250 |loss: 0.19000036 |
|epoch: 8280 |loss: 0.19078821 |
|epoch: 8310 |loss: 0.1905069 |
|epoch: 8340 |loss: 0.18956289 |
|epoch: 8370 |loss: 0.19050853 |
|epoch: 8400 |loss: 0.18970998 |
|epoch: 8430 |